In [1]:
"""
Notebook for taking the DNN trained in classifier_pretrained.ipynb and fine-tuning
layers on final location data only.
"""
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def accuracy_f_score(y_pred,y_true):
    print(f"Accuracy score: {round(accuracy_score(y_true, y_pred) * 100,2)}%")
    print('\033[92m' + f"F1 score: {f1_score(y_true, y_pred)}" + '\033[0m')
    

def in_city(x_pred,y_pred):
    targets = []
    
    if (3750901.5068 <= x_pred <= 3770901.5069) and (-19268905.6133 <= y_pred <= -19208905.6133):
        return 1
    else:
        return 0

def sigmoid(x):
    e = np.exp(1)
    y = 1/(1+e**(-x))
    return y


def journey_time(x,y):
    """
    Compute journey time in seconds.
    """
    x = pd.to_datetime(x)
    y = pd.to_datetime(y)
    return (y-x).total_seconds()

def to_binary(x):
    result = []
    for n in x:
        result.append(np.argmax(n))
    return result

In [66]:
"""
Load data pertaining to final locations only.
"""
df = pd.read_csv('data_train/final_locations.csv')

df["dpc"] = list(map(sigmoid,df["dist_pct_ch"])) #addresses skewness
df.set_index("trajectory_id", inplace=True)
df["final_loc"] = list(map(in_city, df["x_exit"], df["y_exit"]))

y = df["final_loc"].values
df.drop(["vmax","vmin","vmean","time_entry","time_exit","hash","dist_pct_ch",
         "x_exit","y_exit"], axis=1, inplace=True)
X = df.values
df.head()

,x_entry,y_entry,dist,net_tr,prev_tr,x_home,y_home,nj,j_time,dpc,final_loc
trajectory_id,,,,,,,,,,,
traj_0000a8602cf2def930488dee7cdad104_1_5,3.744945e+06,-1.928183e+07,45797.982227,99463.898797,3544.948847,3.751014e+06,-1.909398e+07,6.0,962.0,0.482047,0
traj_0000cf177130469eeac79f67b6bcf3df_9_3,3.749088e+06,-1.926605e+07,29603.985176,-1056.813994,270.043451,3.749450e+06,-1.926506e+07,4.0,1756.0,0.497740,0
traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,3.758738e+06,-1.937594e+07,137051.659155,-1867.319643,-1867.319643,3.771461e+06,-1.910413e+07,2.0,2716.0,0.503453,0
traj_0002124248b0ca510dea42824723ccac_31_10,3.767866e+06,-1.917797e+07,61336.955341,5460.552001,-59655.060438,3.765544e+06,-1.917227e+07,9.0,0.0,1.000000,0
traj_000219c2a6380c307e8bffd85b5e404b_23_16,3.747641e+06,-1.922695e+07,17851.785279,-7113.420678,0.000000,3.760336e+06,-1.922818e+07,8.0,0.0,0.500000,0


In [41]:
df = df.apply(abs,axis=1)
df = df.apply(np.log10,axis=1)
df[df == -np.inf] = 0.0
df.head()

,x_entry,y_entry,dist,net_tr,prev_tr,x_home,y_home,nj,j_time,dpc
trajectory_id,,,,,,,,,,
traj_0000a8602cf2def930488dee7cdad104_1_5,6.573445,7.285148,4.660846,4.997665,3.549610,6.574149,7.280896,0.778151,2.983175,-3.169107e-01
traj_0000cf177130469eeac79f67b6bcf3df_9_3,6.573926,7.284793,4.471350,3.023999,2.431434,6.573968,7.284770,0.602060,3.244525,-3.029973e-01
traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,6.575042,7.287263,5.136884,3.271219,3.271219,6.576510,7.281127,0.301030,3.433930,-2.980409e-01
traj_0002124248b0ca510dea42824723ccac_31_10,6.576095,7.282803,4.787722,3.737237,4.775647,6.575828,7.282674,0.954243,0.000000,-1.928655e-16
traj_000219c2a6380c307e8bffd85b5e404b_23_16,6.573758,7.283910,4.251682,3.852078,0.000000,6.575227,7.283938,0.903090,0.000000,-3.010300e-01


In [4]:
scaler = MinMaxScaler(feature_range=(-1,1))
X = df.values
#X = scaler.fit_transform(X)

In [20]:
from keras.callbacks import LearningRateScheduler
import math
def step_decay(epoch):
    initial_lrate = 0.0005
    drop = 0.25
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)

In [24]:
adam = keras.optimizers.Adam(lr=.0, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
NN_model = load_model('pretrained_model_wed.h5')
#NN_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [25]:
"""
Freeze layers for fine-tuning -- 
training on all but the input layer(s) better than training on all layers or [-5].
"""
for layer in NN_model.layers[:-9]:
    layer.trainable = False
    
for layer in NN_model.layers:
    print(layer, layer.trainable)

<keras.layers.core.Dense object at 0x0000014F69A1BCC0> False
<keras.layers.advanced_activations.LeakyReLU object at 0x0000014F69A545F8> False
<keras.layers.normalization.BatchNormalization object at 0x0000014F69A54518> False
<keras.layers.core.Dropout object at 0x0000014F69A545C0> False
<keras.layers.core.Dense object at 0x0000014F69A5BF28> False
<keras.layers.advanced_activations.LeakyReLU object at 0x0000014F69A54DA0> False
<keras.layers.normalization.BatchNormalization object at 0x0000014F71D9DB38> False
<keras.layers.core.Dropout object at 0x0000014F71E16D30> True
<keras.layers.core.Dense object at 0x0000014F71ECBBE0> True
<keras.layers.advanced_activations.LeakyReLU object at 0x0000014F71F22FD0> True
<keras.layers.normalization.BatchNormalization object at 0x0000014F71DE5CF8> True
<keras.layers.core.Dropout object at 0x0000014F71F4B160> True
<keras.layers.core.Dense object at 0x0000014F7202BE10> True
<keras.layers.advanced_activations.LeakyReLU object at 0x0000014F71F5E438> True
<

In [26]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint,loss_history,lrate]
class_weight = {0: .5,
                1: .66}
y_binary = to_categorical(y)
x_train,x_test,y_train,y_test = train_test_split(X,y_binary,test_size=0,random_state=410,shuffle=True)

NN_model.fit(x_train, y_train, epochs=100, batch_size=256, validation_split = 0.2,
             callbacks=callbacks_list, class_weight=class_weight)

Train on 107229 samples, validate on 26808 samples
Epoch 1/100
107229/107229 [==============================] - 48s 449us/step - loss: 0.0902 - acc: 0.9375 - val_loss: 0.0897 - val_acc: 0.9356

Epoch 00001: val_loss improved from inf to 0.08967, saving model to Weights-001--0.08967.hdf5
Epoch 2/100
107229/107229 [==============================] - 49s 460us/step - loss: 0.0894 - acc: 0.9379 - val_loss: 0.0889 - val_acc: 0.9377

Epoch 00002: val_loss improved from 0.08967 to 0.08891, saving model to Weights-002--0.08891.hdf5
Epoch 3/100
107229/107229 [==============================] - 50s 470us/step - loss: 0.0893 - acc: 0.9379 - val_loss: 0.0894 - val_acc: 0.9371

Epoch 00003: val_loss did not improve
Epoch 4/100
107229/107229 [==============================] - 47s 441us/step - loss: 0.0893 - acc: 0.9383 - val_loss: 0.0892 - val_acc: 0.9363

Epoch 00004: val_loss did not improve
Epoch 5/100
107229/107229 [==============================] - 49s 458us/step - loss: 0.0886 - acc: 0.9387 - va

KeyboardInterrupt: 

In [27]:
weights_file = 'Weights-053--0.08802.hdf5' # choose the best checkpoint 
NN_model.load_weights(weights_file) # load it
NN_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [28]:
"""Make predictions if using only a sub-sample of the training data"""

X_test = df_test.values
# X_test = scaler.fit_transform(X_test)

pred = NN_model.predict(X_test)
pred_b = to_binary(pred)
ids = df_test.index
results_df = pd.DataFrame()
results_df["id"] = ids
results_df["target"] = pred_b

results_df.to_csv("../results/classifier_TL_b1.csv", index=False)